In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


In [38]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx
import h5py
#import obonet
from Bio import SeqIO
import re
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import gc
import pickle
import psutil
import torch
import joblib
from transformers import T5EncoderModel, T5Tokenizer
#from tape import ProteinBertModel, UniRepModel, TAPETokenizer

Getting the protein sequence from fasta files

In [39]:
id = []
seq = [] 

for seq_record in SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta", "fasta"):
    id.append(seq_record.id)
    seq.append(str(seq_record.seq))

print(len(id))

142246


In [40]:
seq_df = pd.DataFrame({"id": id, "sequence": seq})
del id
del seq
seq_df.head()

,id,sequence
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
1,O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
2,O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
3,A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
4,P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...


In [41]:
gc.collect()

0

In [42]:
seq_df = seq_df.sort_values("id").reset_index().drop("index", axis = 1)
seq_df.head()

,id,sequence
0,A0A009IHW8,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,A0A021WW32,MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQ...
2,A0A021WZA4,MKYINCTQPAIDDFPRDLFSEAQRQSGAVVLHVIASLYLFVALAVV...
3,A0A023FBW4,MTSHGAVKIAIFAVIALHSIFECLSKPQILQRTDHSTDSDWDPQMC...
4,A0A023FBW7,MKVLLYIAASCLMLLALNVSAENTQQEEEDYDYGTDTCPFPVLANK...


**Extract other features**

In [43]:
lst = []
for s in seq_df["sequence"]:
    lst.append(ProteinAnalysis(s).get_amino_acids_percent())
X = pd.DataFrame(lst)
X.head()

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,0.040892,0.000000,0.074349,0.085502,0.014870,0.029740,0.018587,0.096654,0.126394,0.107807,0.011152,0.055762,0.022305,0.055762,0.033457,0.081784,0.059480,0.048327,0.011152,0.026022
1,0.061611,0.006319,0.078989,0.082148,0.045814,0.058452,0.033175,0.067930,0.056872,0.091627,0.025276,0.067930,0.048973,0.030016,0.036335,0.082148,0.045814,0.056872,0.006319,0.017378
2,0.110220,0.032064,0.038076,0.038076,0.052104,0.076152,0.012024,0.086172,0.042084,0.094188,0.030060,0.038076,0.036072,0.022044,0.020040,0.068136,0.070140,0.088176,0.016032,0.030060
3,0.061856,0.072165,0.082474,0.041237,0.030928,0.092784,0.051546,0.061856,0.030928,0.061856,0.030928,0.041237,0.041237,0.041237,0.010309,0.113402,0.072165,0.041237,0.020619,0.000000
4,0.059322,0.076271,0.059322,0.093220,0.016949,0.076271,0.008475,0.008475,0.101695,0.093220,0.033898,0.059322,0.033898,0.033898,0.033898,0.025424,0.059322,0.067797,0.016949,0.042373


In [44]:
del lst

In [45]:
train_terms = pd.read_table("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv")
train_terms.head()

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [46]:
# get most occured label
num_label = 1500

freqCount = (train_terms['term'].value_counts())
print(freqCount)
considered_one = list(freqCount.index[:num_label])

GO:0005575    92912
GO:0008150    92210
GO:0110165    91286
GO:0003674    78637
GO:0005622    70785
              ...  
GO:0031772        1
GO:0042324        1
GO:0031771        1
GO:0051041        1
GO:0102628        1
Name: term, Length: 31466, dtype: int64


In [47]:
# make multilabel data
train_size = len(seq_df)
Y = np.zeros((train_size ,num_label))
train_protein = pd.Series(seq_df["id"])
train_terms_smaller = train_terms[train_terms["term"].isin(considered_one)]
for i in tqdm(range(Y.shape[1])):
    m = train_terms_smaller['term'] ==  considered_one[i]
    Y[:,i] =  train_protein.isin( set(train_terms_smaller[m]['EntryID'] ) ).astype(float )
Y

100%|██████████| 1500/1500 [20:32<00:00,  1.22it/s]


array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [48]:
Y.shape

(142246, 1500)

In [49]:
X = X.values
X.shape

(142246, 20)

In [50]:
del train_terms
del freqCount

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 101)

In [52]:
from sklearn.metrics import f1_score

**Binary Relevance Naive Bayes**

In [53]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

In [54]:
br_gnb = BinaryRelevance(GaussianNB())

In [55]:
br_gnb.fit(X_train, Y_train)
# 25 min to train if use embedding
# 1 min to train if use protein occurence

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [56]:
pred_br_gnb = br_gnb.predict(X_test)
# 19 min to test if use embedding
# 1 min to test if use protein occurence

In [57]:
print(f1_score(Y_test,pred_br_gnb.toarray(), average="micro")) 
print(f1_score(Y_test,pred_br_gnb.toarray(), average="macro"))

0.24561923391232063
0.055964209628973284


Label Powerset Naive Bayes

In [58]:
from skmultilearn.problem_transform import LabelPowerset

In [59]:
lp_gnb = LabelPowerset(GaussianNB())

In [60]:
lp_gnb.fit(X_train, Y_train)

LabelPowerset(classifier=GaussianNB(), require_dense=[True, True])

In [1]:
pred_lp_gnb = lp_gnb.predict(X_test)

NameError: name 'lp_gnb' is not defined

In [ ]:
print(f1_score(Y_test,pred_lp_gnb.toarray(), average="micro")) 
print(f1_score(Y_test,pred_lp_gnb.toarray(), average="macro"))

In [61]:
del X
del Y
del X_train
del Y_train
del seq_df

Try on test data


In [62]:
test_id = []
test_seq = []

for seq_record in SeqIO.parse("/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta", "fasta"):
    test_id.append(seq_record.id)
    test_seq.append(str(seq_record.seq))

print(len(test_id))

141865


In [63]:
test_df = pd.DataFrame({"id": test_id, "sequence": test_seq})
del test_id
del test_seq
test_df.head()

,id,sequence
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...


In [64]:
test_df = test_df.sort_values("id").reset_index().drop("index", axis = 1)
test_df.head()

,id,sequence
0,A0A023PXA5,MLLSELVATASSLPYTAISIHNNCRVPAARHIHHGCRYFHGPPVMH...
1,A0A023PXB0,MFINGFVNYPVRTPPNDLLQVVLHGFLRCPLDGSQVDSIGIGHTVH...
2,A0A023PXB5,MFALIISSKGKTSGFFFNSSFSSSALVGIAPLTAYSALVTPVFKSF...
3,A0A023PXB9,MEYVLIYNIWFFSFLQDKPCFCFVDYACSIFLLSSYCGNCLTAVAT...
4,A0A023PXC2,MLPLCLTFLSFFLSLGGSFKAVMTKEEADGTTEAAACLFWIFNWTV...


In [65]:
id_lst = test_df["id"].tolist()

Make test data and remove all those unneeded or write the needed into disk in order to prevent ram overloading

In [66]:
test_lst = []
for s in test_df["sequence"]:
    test_lst.append(ProteinAnalysis(s).get_amino_acids_percent())
test = pd.DataFrame(test_lst)
test = test.values

In [67]:
del test_df
del test_lst

In [68]:
test.shape

(141865, 20)

In [69]:
test_sub1 = test[:30000]
test_sub2 = test[30000:60000]
test_sub3 = test[60000:90000]
test_sub4 = test[90000:120000]
test_sub5 = test[120000:]
del test

In [70]:
prob_1 = br_gnb.predict(test_sub1)
del test_sub1
with open('prob_1.pickle', 'wb') as f:
    pickle.dump(prob_1, f)
del prob_1

In [71]:
gc.collect()

0

In [72]:
prob_2 = br_gnb.predict(test_sub2)
del test_sub2
with open('prob_2.pickle', 'wb') as f:
    pickle.dump(prob_2, f)
del prob_2

In [73]:
gc.collect()

0

In [74]:
prob_3 = br_gnb.predict_proba(test_sub3)
del test_sub3
with open('prob_3.pickle', 'wb') as f:
    pickle.dump(prob_3, f)
del prob_3

In [75]:
gc.collect()

0

In [76]:
prob_4 = br_gnb.predict_proba(test_sub4)
del test_sub4
with open('prob_4.pickle', 'wb') as f:
    pickle.dump(prob_4, f)
del prob_4

In [77]:
gc.collect()

0

In [78]:
prob_5 = br_gnb.predict_proba(test_sub5)
del test_sub5
with open('prob_5.pickle', 'wb') as f:
    pickle.dump(prob_5, f)
del prob_5

In [79]:
gc.collect()

0

Now concat the data with the label

In [80]:
final_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [81]:
final_df["id"] = [id_lst[i] for i in range(30000) for _ in range(1500)]

In [82]:
final_df["terms"] = considered_one * 30000

In [83]:
with open('prob_1.pickle', 'rb') as f:
    prob_1 = pickle.load(f)
prob_1.shape

(30000, 1500)

In [84]:
prob_1 = prob_1.toarray().ravel()

In [85]:
final_df["prob"] = prob_1

In [86]:
del prob_1

In [87]:
final_df = final_df[final_df["prob"] >= 0.6]

In [88]:
final_df.shape

(1000181, 3)

In [89]:
gc.collect()

0

In [90]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [91]:
temp_df["id"] = [id_lst[i] for i in range(30000, 60000) for _ in range(1500)]

In [92]:
temp_df["terms"] = considered_one * 30000

In [93]:
with open('prob_2.pickle', 'rb') as f:
    prob_2 = pickle.load(f)

In [94]:
prob_2 = prob_2.toarray().ravel()

In [95]:
temp_df["prob"] = prob_2

In [96]:
del prob_2

In [97]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [98]:
temp_df.shape

(956299, 3)

In [99]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [100]:
gc.collect()

0

In [101]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [102]:
temp_df["id"] = [id_lst[i] for i in range(60000, 90000) for _ in range(1500)]

In [103]:
temp_df["terms"] = considered_one * 30000

In [104]:
with open('prob_3.pickle', 'rb') as f:
    prob_3 = pickle.load(f)

In [105]:
prob_3 = prob_3.toarray().ravel()

In [106]:
temp_df["prob"] = prob_3

In [107]:
del prob_3

In [108]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [109]:
temp_df.shape

(761544, 3)

In [110]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [111]:
gc.collect()

0

In [112]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [113]:
temp_df["id"] = [id_lst[i] for i in range(90000, 120000) for _ in range(1500)]

In [114]:
temp_df["terms"] = considered_one * 30000

In [115]:
with open('prob_4.pickle', 'rb') as f:
    prob_4 = pickle.load(f)

In [116]:
prob_4 = prob_4.toarray().ravel()

In [117]:
temp_df["prob"] = prob_4


In [118]:
del prob_4

In [119]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [120]:
temp_df.shape

(717937, 3)

In [121]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [122]:
gc.collect()

0

In [123]:
temp_df = pd.DataFrame(columns = ["id", "terms", "prob"])

In [124]:
temp_df["id"] = [id_lst[i] for i in range(120000, 141865) for _ in range(1500)]

In [125]:
temp_df["terms"] = considered_one * 21865

In [126]:
with open('prob_5.pickle', 'rb') as f:
    prob_5 = pickle.load(f)

In [127]:
prob_5 = prob_5.toarray().ravel()

In [128]:
temp_df["prob"] = prob_5

In [129]:
del prob_5

In [130]:
temp_df = temp_df[temp_df["prob"] >= 0.6]

In [131]:
temp_df.shape

(526417, 3)

In [132]:
final_df = pd.concat([final_df, temp_df])
del temp_df
final_df = final_df.reset_index().drop("index", axis = 1)

In [133]:
final_df.shape

(3962378, 3)

In [134]:
# make the submission
final_df.to_csv("/kaggle/working/submission.tsv", index = False, sep = "\t")